In [1]:
'''GOOD VERSION TO USE'''

import POIO_1st as po
# import POIO as po

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
from lmfit.models import GaussianModel, LorentzianModel, VoigtModel, ExponentialModel
import sys
import lmfit
import scipy as sp
import os


Version = 'old'
# Version = 'new'

# ou sont les données ... 
# Dir      = '/home/jbvielfaure/Desktop/JEREMY/DATA_simu/HI_D_SMC/00161/star_best/direction100_100k/'
# Dir      = '/home/jbvielfaure/Desktop/JEREMY/DATA_simu/HI_D_SMC/00161/star_best/direction100_100k_161_old/'
Dir      = '/home/jbvielfaure/Desktop/JEREMY/DATA_simu/HI_D_SMC/00164/star_best/direction100_100k2/'
FileName = 'LyaPhotIC_100k.recLya.result.1' 
    # lecture des mocks
ncpu     = 24   # nb de cpus utilisés pour le calcul
ICFile = 'LyaPhotIC_100k.recLya.1'
NbDir    = 100    # nb de directions analysees
    # Save plots and data
NameImage   = 'image_100k_'
NameSpec    = 'spectre_100k_'
NameVal     = 'PhyData'

    # lecture fichier donnees
## DATA
# path        = '/home/jbvielfaure/Desktop/JEREMY/DATA_simu/HI_D_SMC/00164/star1/...'
# path        = '/home/jbvielfaure/Desktop/JEREMY/DATA_simu/HI_D_SMC/00161/star_best/'
path          = '/home/jbvielfaure/Desktop/JEREMY/DATA_simu/HI_D_SMC/00164/star_best/'

file        = 'LOS100S.dat'

with open(path + file) as f:
    lines = f.readlines()

NHI  = np.loadtxt(path + file, usecols=(6,), unpack=True, skiprows=0,)
NHI0 = np.log10(NHI)
NHI0 = np.array(np.around(NHI0, decimals=2))
# print(NHI)

# unités etc correspondant au pas de temps utilisé
# timeS    = [164, 163, 162, 161]
step     = 0
# redshift =  1./0.250002220774516-1.
red_shift = [3.000, 3.013, 3.025, 3.038]
# convertit les longueurs en cm 
unit_long   = [0.229476043567044e+26, 0.22875660056933e+26, 0.228032810185619e+26, 0.227305981266731e+26]
redshift = red_shift[step]
unit_l   = unit_long[step]
# print(redshift)

# les constantes ... 
pc2cm = 3.086e18
cm2pc = 1./pc2cm

# la cosmologie ... 
from astropy.cosmology import FlatLambdaCDM, z_at_value
import astropy.units as u
cosmo = FlatLambdaCDM(H0=67.11,Om0=0.3175)
# angular distance scale and luminosity distance
a =  cosmo.arcsec_per_kpc_proper(redshift)
unit_l_pkpc = unit_l * cm2pc / 1000. # converts comoving code units to phys. kpc
unit_l_arcsec = unit_l_pkpc * a.value
lumDist    = cosmo.luminosity_distance(redshift)
lumDist_cm = lumDist.cgs.value

## Open file to record physical info of the line
os.makedirs(os.path.dirname(Dir + 'DATA/'), exist_ok=True)
fill_val = open(Dir + 'DATA/' + NameVal + '.dat', "w")
# fill_val.write('# %s\t %s\t %s\t %s\t %s\t %s\t %s\t %s\t %s\t %s\t %s\t %s\t %s\t %s\t %s\n' %('ID', 'NHI', 'Vpeak', 'VpeakG', 'Imax', 'ImaxG', 'Flux', 'x', 'y', 'z', 'kx', 'ky', 'kz', 'NHI', 'r'))
fill_val.write('# %s\t\t %s\t\t %s\t\t %s\t\t %s\t\t %s\t\t %s\n' %('ID', 'NHI', 'Vpeak', 'VpeakG', 'Imax', 'ImaxG', 'Flux'))

## Initialisation properties 
Vpeak, VpeakG, Imax, ImaxG, Flux, FWHM = ([] for i in range(6))

# lecture des mocks
# Boucle sur le nb de directions
for id_ind in np.arange(NbDir):
#     idir = 1

    # NHI value of the direction for the name
    NHI0str  = str(NHI0[id_ind])
    NHI0_str = NHI0str.replace(".", "")

    if Version == 'old':
        mock = po.mockobs(Dir,FileName,ncpu,load_cube=True,unit_l_arcsec=unit_l_arcsec,lumdist_cm=lumDist_cm,redshift=redshift,idirection=id_ind+1)
    elif Version == 'new':
        mock = po.mockobs(Dir,FileName,ICFile,load_cube=True,unit_l_arcsec=unit_l_arcsec,lumdist_cm=lumDist_cm,redshift=redshift,idirection=id_ind+1)

    
    
    ## IMAGE
    image = np.sum(mock.cube,axis=2) * mock.cube_dlbda_Angstrom
    smooth_scale_pix = 2  # un petit smoothing ... 
    if smooth_scale_pix > 0:
        import scipy.ndimage as ndimage
        image = ndimage.gaussian_filter(image,sigma=(smooth_scale_pix,smooth_scale_pix))

    plt.figure(figsize=(5,5))
    ax = plt.subplot(111)
    # im = plt.imshow(image,norm=LogNorm(),vmin=image.max()/1e3,vmax=image.max(),origin='lower', \
    #            extent=[mock.cube_xmin_arcsec,mock.cube_xmax_arcsec,mock.cube_xmin_arcsec,mock.cube_xmax_arcsec],\
    #            cmap='Greys')
    im = plt.imshow(image,norm=LogNorm(),vmin=1e-21,vmax=1e-17,origin='lower', \
               extent=[mock.cube_xmin_arcsec,mock.cube_xmax_arcsec,mock.cube_xmin_arcsec,mock.cube_xmax_arcsec],\
               cmap='Greys')
    plt.xlabel('arcsec')
    plt.ylabel('arcsec')
    # la color bar 
    axins = inset_axes(ax,width="100%",height="3%",loc=1,borderpad=0)
    cbar  = plt.colorbar(im,cax=axins,orientation='horizontal',label=r'$erg/s/cm^2/arcsec^2$')

    os.makedirs(os.path.dirname(Dir + 'IMAGEs/'), exist_ok=True)
#     plt.savefig(Dir + 'IMAGEs/' + NameImage + str(id_ind) + '_' + NHI0_str + '.pdf')
    plt.savefig(Dir + 'IMAGEs/' + NameImage + str(id_ind) + '_' + NHI0_str + '.png')
    
    
    
    ## SPECTRE
    spectrum = np.sum(mock.cube,axis=(0,1)) * mock.cube_dx_arcsec * mock.cube_dx_arcsec
    plt.figure(figsize=(7,5))
    lambda0 = (1+redshift)*1215.67
    # v = (mock.cube_lbda_Angstrom - ((1+redshift)*1215.67))/((1+redshift)*1215.67)*299792.458
    v = 299792.458*((mock.cube_lbda_Angstrom - lambda0)/lambda0)

    ## WRITE DATA
    os.makedirs(os.path.dirname(Dir + 'DATA/'), exist_ok=True)
    fill_txt = open(Dir + 'DATA/' + NameSpec + str(id_ind) + '_' + NHI0_str + '.dat', "w")
    fill_txt.write('#' + lines[id_ind+1])
    for val in range(len(mock.cube_lbda_Angstrom)):
        fill_txt.write('%.6e %.6e\n' % (mock.cube_lbda_Angstrom[val], spectrum[val]))
    fill_txt.close()

    plt.plot(v,spectrum, label= 'NHI = '+NHI0str)
    # plt.plot(mock.cube_lbda_Angstrom,spectrum)
    plt.plot([0,0],[9e-20, 0], '--', alpha=0.3, color='gray')
    # plt.xlabel(r'$\lambda_{\rm obs} \ [\AA]$',fontsize=15)
    plt.xlabel(r'$velocity \ [km.s^{-1}]$',fontsize=15)
    plt.ylabel(r'$F \ [erg \ s^{-1}\ cm^{-2}\ \AA^{-1}]$',fontsize=15)
    plt.legend()
#     plt.axvline(1215.67*(1.+redshift),color='red',linewidth=0.5)
    
    os.makedirs(os.path.dirname(Dir + 'SPECTREs/'), exist_ok=True)
#     plt.savefig(Dir + 'SPECTREs/' + NameSpec + str(id_ind) + '_' + NHI0_str + '.pdf')
    plt.savefig(Dir + 'SPECTREs/' + NameSpec + str(id_ind) + '_' + NHI0_str + '.png')
    
    
    ## DETERMINATION PROPERTIES OF THE LINE
    
    ### Velocity for the maximal intensity
    max_value = np.amax(spectrum) # max(spectrum)
    max_index = np.where(spectrum == max_value) # spectrum.index(max_value)
    max_v     = v[max_index]
    Vpeak.append(max_v)
    
    ### Max intensity
    Imax.append(max_value)
    
    ### v of the max for gaussian fit
    #### Gaussian fit
    mod = GaussianModel()
    pars = mod.guess(spectrum*1e20, x=v)
    result = mod.fit(spectrum*1e20, pars, x=v)
    
    os.makedirs(os.path.dirname(Dir + 'Results_fit/'), exist_ok=True)
    F = open(Dir + 'Results_fit/' + 'results_fit_v_1G_' + str(id_ind) + '.dat', 'w')
    F.write(result.fit_report())
    F.close
    
    os.makedirs(os.path.dirname(Dir + 'FIT/'), exist_ok=True)
    f1 = open(Dir + 'FIT/' + 'Fit_1g_' + str(id_ind) + '.dat', 'w')
    fit_result = np.column_stack((v, result.best_fit*1e-20))
    for item in np.arange(len(v)):
        f1.write("%.2f %.6e\n" % (fit_result[item, 0], fit_result[item, 1]))
    f1.close()
        
    #### Max
    max_valueG = np.amax(result.best_fit*1e-20)
    max_indexG = np.where(result.best_fit*1e-20 == max_valueG)
    max_vG     = v[max_indexG]
    VpeakG.append(max_vG)
    
    ### Max intensity for gaussian fit
    
    ImaxG.append(max_valueG) 

    ### Flux exact
    ## Flux Simpson ############################################
    integrated_flux = sp.integrate.simps(spectrum,mock.cube_lbda_Angstrom)
    Flux.append(integrated_flux)

    ### FWHM from fit by simple gaussian
#     FWHM = 
#     FWHM.append()
    
    ## TABLE WITH INFO : 'ID', 'NHI', 'Vpeak', 'VpeakG', 'Imax', 'ImaxG', 'Flux', 'FWHM'  
#     print (id_ind)
#     print (NHI0[id_ind])
#     print (Vpeak[id_ind])
#     print (VpeakG[id_ind])
#     print (Imax[id_ind])
#     print (ImaxG[id_ind])
#     print (Flux[id_ind])
    
    fill_val.write('%i\t %8.2f\t %8.2f\t %8.2f\t %8.2e\t %8.2e\t %8.2e\n' %(id_ind, NHI0[id_ind], Vpeak[id_ind], VpeakG[id_ind], Imax[id_ind], ImaxG[id_ind], Flux[id_ind]))

fill_val.close()
    
    
## PLOT/SAVE PROPERTIES OF THE LINE

###1 NHI Vpeak
fig = plt.figure(tight_layout=True, figsize=(7, 5))
ax = fig.add_subplot(111)
# plt.figure(figsize=(7,5))
ax.plot(NHI0, Vpeak, linestyle = 'None', color='darkblue', marker ='*', label='', markersize=12)
plt.xlabel(r'$  \rm log(NHI/cm^{-2})$',fontsize=15)
plt.ylabel(r'$  \rm Vpeak \ [km.s^{-1}]$',fontsize=15)
# plt.legend()
os.makedirs(os.path.dirname(Dir + 'DATA/'), exist_ok=True)
# plt.savefig(Dir + 'DATA/' + 'NHI_Vpeak.pdf')
plt.savefig(Dir + 'DATA/' + 'NHI_Vpeak.png')

###2 NHI VpeakG
plt.figure(figsize=(7,5))
plt.plot(NHI0, VpeakG, linestyle = 'None', color='darkblue', marker ='*', label='', markersize=12)
plt.xlabel(r'$  \rm log(NHI/cm^{-2})$',fontsize=15)
plt.ylabel(r'$  \rm VpeakG \ [km.s^{-1}]$',fontsize=15)
# plt.legend()
os.makedirs(os.path.dirname(Dir + 'DATA/'), exist_ok=True)
# plt.savefig(Dir + 'DATA/' + 'NHI_VpeakG.pdf')
plt.savefig(Dir + 'DATA/' + 'NHI_VpeakG.png')

###3 Vpeak VpeakG
plt.figure(figsize=(7,5))
plt.plot(Vpeak,VpeakG, linestyle = 'None', color='darkblue', marker ='*', label='', markersize=12)
plt.xlabel(r'$  \rm Vpeak \ [km.s^{-1}]$',fontsize=15)
plt.ylabel(r'$  \rm VpeakG \ [km.s^{-1}]$',fontsize=15)
# plt.legend()
os.makedirs(os.path.dirname(Dir + 'DATA/'), exist_ok=True)
# plt.savefig(Dir + 'DATA/' + 'Vpeak_VpeakG.pdf')
plt.savefig(Dir + 'DATA/' + 'Vpeak_VpeakG.png')

###4 NHI Imax
plt.figure(figsize=(7,5))
plt.plot(NHI0, Imax, linestyle = 'None', color='darkblue', marker ='*', label='', markersize=12)
plt.xlabel(r'$  \rm log(NHI/cm^{-2})$',fontsize=15)
plt.ylabel(r'$  \rm Intensity\ max\ [erg\ s^{-1}\ cm^{-2}\ \AA^{-1}]$',fontsize=15)
# plt.legend()
os.makedirs(os.path.dirname(Dir + 'DATA/'), exist_ok=True)
# plt.savefig(Dir + 'DATA/' + 'NHI_Imax.pdf')
plt.savefig(Dir + 'DATA/' + 'NHI_Imax.png')

###5 NHI ImaxG
plt.figure(figsize=(7,5))
plt.plot(NHI0, ImaxG, linestyle = 'None', color='darkblue', marker ='*', label='', markersize=12)
plt.xlabel(r'$  \rm log(NHI/cm^{-2})$',fontsize=15)
plt.ylabel(r'$  \rm Intensity\ max G\ [erg\ s^{-1}\ cm^{-2}\ \AA^{-1}]$',fontsize=15)
# plt.legend()
os.makedirs(os.path.dirname(Dir + 'DATA/'), exist_ok=True)
# plt.savefig(Dir + 'DATA/' + 'NHI_ImaxG.pdf')
plt.savefig(Dir + 'DATA/' + 'NHI_ImaxG.png')

###6 Imax ImaxG
plt.figure(figsize=(7,5))
plt.plot(Imax,ImaxG, linestyle = 'None', color='darkblue', marker ='*', label='', markersize=12)
plt.xlabel(r'$  \rm Intensity\ max\ [erg\ s^{-1}\ cm^{-2}\ \AA^{-1}]$',fontsize=15)
plt.ylabel(r'$  \rm Intensity\ maxG\ [erg\ s^{-1}\ cm^{-2}\ \AA^{-1}]$',fontsize=15)
# plt.legend()
os.makedirs(os.path.dirname(Dir + 'DATA/'), exist_ok=True)
# plt.savefig(Dir + 'DATA/' + 'Imax_ImaxG.pdf')
plt.savefig(Dir + 'DATA/' + 'Imax_ImaxG.png')

###7 NHI Flux
plt.figure(figsize=(7,5))
plt.plot(NHI0, Flux, linestyle = 'None', color='darkblue', marker ='*', label='', markersize=12)
plt.xlabel(r'$  \rm log(NHI/cm^{-2})$',fontsize=15)
plt.ylabel(r'$  \rm Flux\ [erg\ s^{-1}\ cm^{-2}]$',fontsize=15)
# plt.legend()
os.makedirs(os.path.dirname(Dir + 'DATA/'), exist_ok=True)
# plt.savefig(Dir + 'DATA/' + 'NHI_Flux.pdf')
plt.savefig(Dir + 'DATA/' + 'NHI_Flux.png')

###8 Vpeak Imax
plt.figure(figsize=(7,5))
plt.plot(Vpeak,Imax, linestyle = 'None', color='darkblue', marker ='*', label='', markersize=12)
plt.xlabel(r'$  \rm Vpeak\ [km.s^{-1}]$',fontsize=15)
plt.ylabel(r'$  \rm Intensity\ max\ [erg\ s^{-1}\ cm^{-2}\ \AA^{-1}]$',fontsize=15)
# plt.legend()
os.makedirs(os.path.dirname(Dir + 'DATA/'), exist_ok=True)
# plt.savefig(Dir + 'DATA/' + 'Imax_Vpeak.pdf')
plt.savefig(Dir + 'DATA/' + 'Imax_Vpeak.png')


/home/jbvielfaure/.local/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
